<a href="https://colab.research.google.com/github/yvessong23/ML-TEAM-15/blob/main/BinBuddiesCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import TensorFlow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns
import torchvision.transforms as transforms #missing run pip install torchvision
import numpy as np
from PIL import Image
import os

In [ ]:
#Randomly generates a new augmented image based on the input

# Random generator
rng1 = np.random.RandomState(seed=123)

def RandAugment(inpt_img:Image.Image) -> Image.Image:

    augment_select = rng1.random_integers(0,5)
    
    match augment_select:

        case 0:
            new_img = inpt_img.transform(Image.Transpose.ROTATE_90)
        case 1:
            new_img = inpt_img.transpose(Image.Transpose.ROTATE_180)
        case 2:
            new_img = inpt_img.transpose(Image.Transpose.ROTATE_270)
        case 3:
            new_img = inpt_img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        case 4:
            new_img = inpt_img.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
        case 5:
            rand_bright = rng1.random_integers(0,100) / 100
            rand_contrast = rng1.random_integers(0,100) / 100
            rand_staturation = rng1.random_integers(0,100) / 100
            rand_hue = rng1.random_integers(0,50) / 100

            color_jitter = transforms.ColorJitter(brightness=rand_bright, contrast=rand_contrast, saturation=rand_staturation, hue=rand_hue)
            
            new_img = color_jitter(inpt_img)
        case _:
            print("Something went wrong")

    return new_img

In [ ]:
%pip install numpy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hashlib, os
from PIL import Image
%pip install kagglehub
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zlatan599/garbage-dataset-classification")
rgb_path = "cleaned/garbage-dataset-rgb"
print("Path to dataset files:", path)

hashes = {}
duplicates = []

def file_hash(filePath):
    with open(filePath, "rb") as f: # Open and read files for hashing
        file_bytes = f.read()
        return hashlib.md5(file_bytes).hexdigest()


for root, dirs, files in os.walk(path):
    
    augment = 0 # if augment is 0, will generate an augmented version of the image current file (ADDED)

    for file in files:

        file_path = os.path.join(root, file)
        h = file_hash(file_path)

        if h in hashes:
            print(f"Duplicate found! Removing {file_path} (same as {hashes[h]})")
            try:
                os.remove(file_path)
                duplicates.append((file_path, hashes[h]))
            except OSError as e:
                print(f"Warning: Could not remove {file_path} due to read-only filesystem. Error: {e}")
        else:
            hashes[h] = file_path
            try:
                # Preserve the dataset’s subfolder structure
                rel_dir = os.path.relpath(root, path)
                save_dir = os.path.join(rgb_path, rel_dir)
                os.makedirs(save_dir, exist_ok=True)

                save_path = os.path.join(save_dir, file)

                # Skip non-image files (like metadata.csv)
                if not file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
                    continue


                augment = (augment + 1) % 2 #Used to determine if current image should get an augmented version (ADDED)

                with Image.open(file_path) as img:

                    if augment == 0:                                # ADDED CODE START
                        file_name, file_extension = file.split('.')
                        new_name = file_name + "_A." + file_extension 

                        augment_img = RandAugment(img)
                        augment_save = os.path.join(save_dir, new_name)

                        augment_img.convert("RGB").save(augment_save)   #ADDED CODE END

                    img.convert("RGB").save(save_path)
                    img = Image.open(file_path)

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Path to dataset files: C:\Users\Foces\.cache\kagglehub\datasets\zlatan599\garbage-dataset-classification\versions\5


C:\Users\Foces\AppData\Local\Temp\ipykernel_25260\1826601181.py:8: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead
  augment_select = rng1.random_integers(0,5)


In [12]:
import os
import numpy as np
import pandas as pd
from PIL import Image

from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

rgb_path = "cleaned/garbage-dataset-rgb"
base_path = os.path.join(rgb_path, "Garbage_Dataset_Classification", "images")
DATASET_DIR = base_path

# Set up class names
class_names = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
num_classes = len(class_names)

# Ensure 'duplicates' is defined. It is expected to be populated by cell WJM9MyG1JvjH.
# If cell WJM9MyG1JvjH hasn't been run, 'duplicates' will be an empty list, and thus
# no files will be filtered as duplicates in this cell.
if 'duplicates' not in globals():
    duplicates = []

# Verify and collect all image paths and their labels
image_paths = []
labels = []

duplicate_filepaths = {f[0] for f in duplicates} # Extract only the file paths from the duplicates list

for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join(base_path, class_name)
    if os.path.exists(class_dir):
        class_images = [f for f in os.listdir(class_dir)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
        print(f"{class_name}: {len(class_images)} images")

        for img_file in class_images:
            full_path = os.path.join(class_dir, img_file)
            if full_path not in duplicate_filepaths: # Only add if not a duplicate
                image_paths.append(full_path)
                labels.append(class_idx)
    else:
        print(f"Warning: {class_dir} does not exist")

cardboard: 1718 images
glass: 2293 images
metal: 1628 images
paper: 1809 images
plastic: 1709 images
trash: 2500 images


# Hyper parameter Testing 

In [13]:
# Hyperparameters for searching with Optuna
BLOCK_NUM = [3,4,5]
CONV_LAYERS = [1, 2, 3]
BATCH_SIZES = [16, 32, 64]
EPOCHS = [10, 20, 30]
CONV_LAYER_SIZE_BASE = [32, 64, 128]
FILTERS_SIZE = [3,5,7]
REGULARIZATION_CONST = [0.001, 0.01, 0.1]
REG_TYPE = ["n/a", "L2", "Dropout", "Both"]


In [14]:
%pip install optuna
import optuna
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
import numpy as np

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SIZE   = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load all validation images into memory, then split into val + test
val_generator_full = val_test_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

val_data = []
val_labels = []

for i in range(len(val_generator_full)):
    x, y = val_generator_full[i]
    val_data.append(x[0])
    val_labels.append(y[0])

val_data = np.array(val_data)
val_labels = np.array(val_labels)

X_val, X_test, y_val, y_test = train_test_split(
    val_data,
    val_labels,
    test_size=0.5,
    random_state=42,
    stratify=np.argmax(val_labels, axis=1)
)

val_generator_data = (X_val, y_val)
test_generator_data = (X_test, y_test)



def build_model(trial):

    block_num   = trial.suggest_categorical("block_num", [3, 4, 5])
    conv_layers = trial.suggest_categorical("conv_layers", [1, 2, 3])
    batch_size  = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs      = trial.suggest_categorical("epochs", [10, 20, 30])
    reg_type    = trial.suggest_categorical("reg_type", ["n/a", "L2", "Dropout", "Both"])

    # example: choose regularizer based on reg_type
    if reg_type in ["L2", "Both"]:
        reg = regularizers.l2(trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True))
    else:
        reg = None

    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5) if reg_type in ["Dropout", "Both"] else 0.0

    # Hyperparameters to tune
    f1 = trial.suggest_categorical("f1", [32, 48, 64])
    f2 = trial.suggest_categorical("f2", [64, 96, 128])
    f3 = trial.suggest_categorical("f3", [128, 192, 256])
    f4 = trial.suggest_categorical("f4", [256, 384, 512])

    d1 = trial.suggest_float("d1", 0.1, 0.4)
    d2 = trial.suggest_float("d2", 0.1, 0.4)
    d3 = trial.suggest_float("d3", 0.2, 0.5)
    d4 = trial.suggest_float("d4", 0.2, 0.5)

    dense_units = trial.suggest_categorical("dense_units", [128, 256, 512])
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    kernel = trial.suggest_categorical("kernel", [3, 5])

    model = models.Sequential([
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

        layers.Conv2D(f1, (kernel, kernel), padding='same', activation='relu'),
        layers.Conv2D(f2, (kernel, kernel), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(d1),
        layers.MaxPool2D(2),

        layers.Conv2D(f3, (kernel, kernel), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(d2),
        layers.MaxPool2D(2),

        layers.Conv2D(f4, (3,3), padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(l2_reg)),
        layers.BatchNormalization(),
        layers.Dropout(d3),
        layers.MaxPool2D(2),

        layers.GlobalAveragePooling2D(),

        layers.Dense(dense_units, activation='relu',
                     kernel_regularizer=regularizers.l2(l2_reg)),
        layers.BatchNormalization(),
        layers.Dropout(d4),

        layers.Dense(6, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found 8163 images belonging to 6 classes.
Found 2328 images belonging to 6 classes.


In [15]:
def objective(trial):
    model = build_model(trial)

    history = model.fit(
        train_generator,
        validation_data=val_generator_data,
        epochs=trial.params["epochs"],
        batch_size=trial.params["batch_size"],
        verbose=0
    )

    return history.history["val_loss"][-1]


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best params:", study.best_params)


[I 2025-11-19 17:27:39,288] A new study created in memory with name: no-name-741c3b2f-67b4-43b6-959a-37c98dbc6195
/Users/mp/ML-TEAM-15/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


# Base Model Testing 

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
import numpy as np

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_SIZE   = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

val_test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load all validation images into memory, then split into val + test
val_generator_full = val_test_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

val_data = []
val_labels = []

for i in range(len(val_generator_full)):
    x, y = val_generator_full[i]
    val_data.append(x[0])
    val_labels.append(y[0])

val_data = np.array(val_data)
val_labels = np.array(val_labels)

X_val, X_test, y_val, y_test = train_test_split(
    val_data,
    val_labels,
    test_size=0.5,
    random_state=42,
    stratify=np.argmax(val_labels, axis=1)
)

val_generator_data = (X_val, y_val)
test_generator_data = (X_test, y_test)

model_gap = models.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # Block 1
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.MaxPool2D(2),

    # Block 2
    layers.Conv2D(128, (5,5), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.MaxPool2D(2),

    # Block 3
    layers.Conv2D(256, (3,3), padding='same', activation='relu',
    kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.35),
    layers.MaxPool2D(2),

    # Block 4
    layers.Conv2D(512, (3,3), padding='same', activation='relu',
    kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.35),
    layers.MaxPool2D(2),

    # Head with GAP
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.50),
    layers.Dense(6, activation='softmax'),
])

# Testing CNN with Global Average Pooling

Using ChatGPT and Kaggle, I found that GAP or Global Average Pooling can significantly reduce the number of parameters. This method compresses each feature map into a single value by averaging its activations (outputs). This reduced our dimensionality from 5+ million to about 56K. This also allows the model to generalize better and train faster.
https://www.kaggle.com/code/faressayah/cifar-10-images-classification-using-cnns-88
https://www.kaggle.com/code/zlatan599/f1-0-97-garbage-classif-final-version-2

The baseline run of this model with no dropout, data augmentation, or hyperparameter tuning was:

* Train Accuracy: ~0.76

* Train Loss: ~0.81

* Best Validation Accuracy: ~0.59 (epoch 27)

* Best Validation Loss: ~1.43

* Test Accuracy: 0.5902

* Test Loss: 1.52

* Training Time: ≈ 24 minutes for 30 epochs on GPU

# Strong classes:

Cardboard: 0.74 F1 (0.71 precision / 0.77 recall)

Paper: 0.68 F1

Trash: 0.64 F1

# Medium:

Glass: 0.61 F1

# Weak:

Plastic: 0.50 F1

Metal: 0.40 F1

In [ ]:
print(tf.config.list_physical_devices("GPU"))

model_gap.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model_gap.summary()

checkpoint = ModelCheckpoint(
    'best_model_gap.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

history_gap = model_gap.fit(
    train_generator,
    validation_data=val_generator_data,
    epochs=30,
    callbacks=[checkpoint]
)

test_loss, test_acc = model_gap.evaluate(
    X_test,
    y_test,
    batch_size=BATCH_SIZE,
    verbose=2
)

# Visualization

Model Evaluation at baseline


---

Comparing GAP vs Flatten Head

In [ ]:
test_images, test_labels = test_generator_data

y_probs = model_gap.predict(test_images)

y_pred = np.argmax(y_probs, axis=1)
y_true = np.argmax(test_labels, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

In [ ]:
# Confusion Matrix Visualization

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title("Confusion Matrix (CNN + GAP)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# ROC Curves

from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(8,6))

for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(y_true == i, y_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f"{class_name} (AUC = {roc_auc:.3f})")

plt.plot([0,1], [0,1], 'k--')
plt.title("ROC Curves (One-vs-Rest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Precision–Recall Curves

from sklearn.metrics import precision_recall_curve

plt.figure(figsize=(8,6))

for i, class_name in enumerate(class_names):
    precision, recall, _ = precision_recall_curve(y_true == i, y_probs[:, i])
    plt.plot(recall, precision, lw=2, label=f"{class_name}")

plt.title("Precision–Recall Curves")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.grid(True)
plt.show()


## Loss Curve Plots

In [ ]:
train_loss = history_gap.history['loss']
val_loss   = history_gap.history['val_loss']
train_acc  = history_gap.history['accuracy']
val_acc    = history_gap.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)

fig, ax = plt.subplots(1, 2, figsize=(15,5))

# LOSS
ax[0].plot(epochs, train_loss, 'b-', label='Training Loss', linewidth=2)
ax[0].plot(epochs, val_loss, 'r-', label='Validation Loss', linewidth=2)
ax[0].set_title("Loss Curve", fontsize=15)
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend()
ax[0].grid(True)

# showing final test loss on chart
ax[0].text(0.95, 0.95, f"Test Loss: {test_loss:.4f}",
           ha='right', va='top', transform=ax[0].transAxes,
           fontsize=12, color='black',
           bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

# ACCURACY
ax[1].plot(epochs, train_acc, 'b-', label='Training Accuracy', linewidth=2)
ax[1].plot(epochs, val_acc, 'r-', label='Validation Accuracy', linewidth=2)
ax[1].set_title("Accuracy Curve", fontsize=15)
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Accuracy")
ax[1].legend()
ax[1].grid(True)


ax[1].text(0.95, 0.95, f"Test Acc: {test_acc:.4f}",
           ha='right', va='top', transform=ax[1].transAxes,
           fontsize=12, color='black',
           bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

plt.show()